In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import time

In [2]:
def record_data(label:str):
	mp_hands = mp.solutions.hands
	mp_draw = mp.solutions.drawing_utils

	cap = cv2.VideoCapture(0)
	start_time = None
	total_time = 25
	landmarks = []

	with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
		while cap.isOpened():
			ret, frame = cap.read()
			if not ret:
				break

			frame = cv2.flip(frame, 1)  # Flip for natural interaction
			rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
			results = hands.process(rgb_frame)

			if results.multi_hand_landmarks:
				if start_time is None:
					print(
						"Hand detected, timer is set for 5 second before recording landmarks."
					)
					print(f"Total recording time will be: {total_time} seconds.")
					start_time = time.time()
				idle_time = time.time() - start_time
				elasped_time = time.time() - start_time
				if elasped_time > total_time + 5:
					break
				if idle_time >= 5:
					for hand_landmarks in results.multi_hand_landmarks:
						mp_draw.draw_landmarks(
							frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
						)

						# Extract (x, y, z) coordinates
						landmark = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
						landmarks.append(landmark)  # Use this data for training

			cv2.imshow("Hand Tracking", frame)
			if cv2.waitKey(1) & 0xFF == ord("q"):
				break

	print(len(landmarks))
	#label = input("Enter a label: ")
	flattened_data = [
		item
		for tuple in [tuple for sublist in landmarks for tuple in sublist]
		for item in tuple
	]

	df = pd.read_csv("train.csv")
	for i in range(0, len(flattened_data), 63):
		df = pd.concat(
			[df, pd.DataFrame([flattened_data[i : i + 63] + [label]], columns=df.columns)],
			ignore_index=True,
		)

	df.to_csv(f"./train_{label}.csv",index=False)


	cap.release()
	cv2.destroyAllWindows()


In [6]:
record_data("phrase")

Hand detected, timer is set for 5 second before recording landmarks.
Total recording time will be: 25 seconds.
1071


C:\Users\ki3n9\AppData\Local\Temp\ipykernel_10268\726846536.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(


In [21]:
df = pd.read_csv("./train_F.csv")

In [17]:
df

,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2,X3,...,X18,Y18,Z18,X19,Y19,Z19,X20,Y20,Z20,label
0,0.872446,0.766787,-1.581168e-07,0.790086,0.742904,0.002101,0.721623,0.688093,0.001611,0.668032,...,0.805114,0.455471,-0.029109,0.766614,0.422215,-0.037485,0.730591,0.410932,-0.042381,C
1,0.301810,0.773049,3.948495e-07,0.366321,0.757619,0.001685,0.403041,0.718876,-0.000149,0.431004,...,0.298949,0.491358,-0.028047,0.333129,0.460379,-0.033760,0.369352,0.453112,-0.035923,C
2,0.873678,0.761805,-1.417700e-07,0.792010,0.740459,-0.000120,0.722490,0.687305,-0.002055,0.668155,...,0.804132,0.453362,-0.031153,0.765842,0.419944,-0.039790,0.729335,0.407964,-0.044668,C
3,0.302600,0.770688,4.004773e-07,0.367251,0.755508,-0.000360,0.404492,0.717731,-0.003907,0.433433,...,0.297797,0.493329,-0.035853,0.331891,0.461299,-0.042170,0.368240,0.452445,-0.044584,C
4,0.874891,0.756247,-1.401483e-07,0.791977,0.735475,-0.000816,0.722373,0.686041,-0.003683,0.668229,...,0.805888,0.450287,-0.034377,0.766522,0.417707,-0.043697,0.728814,0.406863,-0.048871,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,0.263018,0.836188,2.983584e-07,0.319230,0.803081,0.013462,0.345781,0.745014,0.018766,0.364050,...,0.263499,0.569212,-0.029684,0.298691,0.544403,-0.036658,0.331123,0.542419,-0.039520,C
1000,0.859321,0.735071,-1.766275e-07,0.791966,0.726355,0.003091,0.728068,0.683971,0.003988,0.682975,...,0.769844,0.481079,-0.025938,0.734019,0.460627,-0.030320,0.703439,0.454762,-0.032721,C
1001,0.256236,0.843068,2.611153e-07,0.315351,0.802969,0.029888,0.339551,0.742110,0.043323,0.358770,...,0.267150,0.566925,-0.020686,0.304045,0.544526,-0.025193,0.335970,0.544223,-0.027319,C
1002,0.859010,0.741641,-2.228855e-07,0.789822,0.730006,0.006697,0.727877,0.684567,0.009448,0.684629,...,0.769434,0.490714,-0.027052,0.731894,0.471007,-0.032234,0.699879,0.466365,-0.035126,C
